# touristAI

This is the second Capstone project as part of the [Artificial Intelligence Nanodegree](https://www.udacity.com/course/artificial-intelligence-nanodegree--nd889), and focuses on Machine Translation Project. It foucses on using multiple variants of RNNs implemented via Keras!

![](http://www.sonicambulance.com/wp-content/uploads/2011/06/Something-French.jpg)


## Introduction

In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Our completed pipeline will accept English text as input and return the French translation!

## Overview:
- **Preprocess** - We'll convert text to sequence of integers.
- **Models** - We'll create models which accepts a **sequence of integers as input** and returns **a probability distribution over possible translations**. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, and **design our own model**!
- **Prediction** - Lastly, we'll run the model on English text, and read our French!

In [2]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import collections

import helper
import numpy as np
import project_tests as tests
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, BatchNormalization, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import load_model

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # Using my local CPU; 🙏 

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14761486045305321065
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline. 

The most common datasets used for machine translation are from [WMT](http://www.statmt.org/):

![](http://www.statmt.org/book/cover.jpg)

However, that will take a long time to train a neural network on, so we'll be using a sample dataset we created for this project; which contains a much smaller vocabulary. It should be possible to actually train our model in a reasonable time with this dataset. 

### Load Data
The data is located in:
* `data/small_vocab_en`: English sentences.
* `data/small_vocab_fr`. Their French translations.

Let's Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Sample Data

Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  

Let's see what we've got:

In [6]:
for sample_i in range(10):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
small_vocab_en Line 3:  california is usually quiet during march , and it is usually hot in june .
small_vocab_fr Line 3:  california est généralement calme en mars , et il est généralement chaud en juin .
small_vocab_en Line 4:  the united states is sometimes mild during june , and it is cold in september .
small_vocab_fr Line 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .
small_vocab_en Line 5:  your least liked fruit is the grape , but my least liked is the apple .
small_vocab_fr Line 5:  votre moins aimé fruit est le raisin , m

From looking at the sentences, you can see they have been preprocessed already:
1. *The puncuations have been delimited using spaces.*
2. *All the text have been converted to lowercase.* 

This should save you some time, (**and are great steps to take in the future**), but the text requires even more preprocessing!

### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary; **A more complex vocabulary is a more complex problem.**  

Let's look at the complexity of the dataset we'll be working with:

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print('\n')
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"


1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words. 

We have therefore traded vocabulary *depth* for optimal translation accuracy, supporting our smaller dictionary with *significantly more examples*.

## Preprocess
For this project, **we won't use text data as input to your model, instead we'll encode our text into sequences of integers**, using the following preprocess methods:

3. *Tokenizing the words into ids.*
4. *Adding padding to make all the sequences the same length.*

Let's go!

### Tokenize 
For a neural network **to predict on text data**, it first **has to be turned into data it can understand.**

Text data like "dog" is actually sequence of ASCII character encodings, stored and encoded in memory: `100 111 103`. 

![](https://www.w3.org/International/articles/definitions-characters/index-data/encodings.png)

Therefore, texts are just a specific format for storage, and can be converted to and from, sort of like a machine translation *inside* our language translator! 

![](https://imgflip.com/s/meme/Yo-Dawg-Heard-You.jpg)

Let's convert *these* numbers into arrays we can run a series of multiplication and addition operations on, and then convert the *output* back into ASCII encoding!

To do this, we need to give each character or each word into a unique ID. These are called **character or word ids**, respectively:
* **Character IDs**: We'd use character ids for *character level models* that generate text predictions for *each character*. 
* **Word IDs**: A *word level model* uses *word ids* that generate text predictions for each word, so we'd need to encode each word as a unique value.

Word level models tend to learn better, since they are lower in complexity, so we'll use one of those.

First, let's turn each sentence into a sequence of words ids using Keras's (built-in!) [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Here's that function running on our `english_sentences` and `french_sentences`:

In [8]:
def tokenize(x):
    """
    Tokenize x
    
    ref: https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/
    
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # fit the tokenizer on the documents
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    return sequences, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'fox': 5, 'lexicography': 15, 'prize': 17, 'this': 18, 'won': 16, 'lazy': 8, 'quick': 2, 'dog': 9, 'brown': 4, 'over': 7, 'short': 20, 'study': 13, 'my': 12, 'of': 14, 'a': 3, 'jove': 11, 'sentence': 21, 'jumps': 6, 'the': 1, 'by': 10, 'is': 19}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding 

When batching the sequence of word ids together, each sequence needs to be the same length, otherwise our linear algebra will trip over misshapen matrices. This is a problem, since **sentences are dynamic in length**, but we can solve it by **adding padding to the end of the sequences to make them the same length**.

As long as we make sure all the English sequences have the same length *and* all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's (built in!) [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function. Let's see how it works: 

In [9]:
def pad(x, length=None):
    """
    Pad input using the "post" option
    e.g. pad([[1, 2, 3]], 5) returns [[1, 2, 3, 0, 0]]
    
    ref: https://keras.io/preprocessing/sequence/
    
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    print(x)
    return pad_sequences(x, maxlen=length, padding="post")

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

[[0, 1, 2, 3], [0, 1, 2, 3, 4, 5], [0, 1, 2]]
[[0, 1, 2, 3], [0, 1, 2, 3, 4, 5], [0, 1, 2]]
[[1, 2, 4, 5, 6, 7, 1, 8, 9], [10, 11, 12, 2, 13, 14, 15, 16, 3, 17], [18, 19, 3, 20, 21]]
Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

Our focus for this project is to build neural network architecture, so we won't create a full preprocess pipeline.  

Instead, we've provided you with the implementation of the `preprocess` function:

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Data Preprocessed


### Post Processing

Brief aside; the neural network will be using our pre-processed tokenized input as words ids, and transformed them around tensor space to get the sequences we want. 

However, this final form would still be a series of word IDs, which isn't what we we want; we need to convert it **using our French dcitionary to convert word IDs back to French.**

The function `logits_to_text` will bridge the gab between the logits (local units) from the neural network to the French translation.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


 We'll also be using this function later to better understand the output of our neural network.

## Models

In this section, we will experiment with various neural network architectures, and see how they do at our problem: **translating English to French using a word-level dictionary.**

We shall do so primarily thanks to the magic of RNNs:

### Recurrent Neural Networks 

So, what are RNNs? Feel free to check out this [this excellent guide](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) for a great review, but the short version is:
* Human memory depends on persistence (i.e. remembering what words mean based on what you've seen.)
* Traditional neural networks were incapable of this, and had no notion of time or memory available; that's where RNN's come in.
* Recurrent neural networks create [little internal loops](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-rolled.png), allowing weights to persist between training sessions.
* [Chaining these little cells](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png) together allows us to encode entire sequences in a very logical way. 
* This has led to some pretty [unexpected effectiveness](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), but we quickly discovered that as traditional RNN has a very short term memory, essentially the sequence itself. 
* Unfortunately, [as this gap grows](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png), RNNS become less and less effective, but a variant called an LSTM (Long Short Term Memory networks) is capable of learning long-term dependencies!
* It does this by essentially **gating its internal memory**, which allows the state to be modified only under careful conditions.

Let's see what we can do with this new **Gated Recurrent Unit** and some example models!

## Models

In this section, we will experiment with various neural network architectures, and see how they do at our problem: **translating English to French using a word-level dictionary.**

We shall do so primarily thanks to the magic of RNNs:

[![](https://i.imgur.com/kpZBDfV.gif)](https://deeplearning4j.org/lstm.html)

### Recurrent Neural Networks 

We will begin by training four relatively simple architectures.
1. **Model 1** is a *basic RNN*
2. **Model 2** is a *RNN with Embedding*
3. **Model 3** is a *Bidirectional RNN*
4. **Model 4** is an *Optional Encoder-Decoder RNN*

After experimenting with the four simple architectures, **we will construct a deeper architecture that is designed to outperform all four models.**



### Hyperparameters

Before we get started, let's define some overall hyperparameters for all of our models, for fair comparison:

In [12]:
# Hyperparameters
learning_rate = 0.05   # 5% Learning rate
batch_size = 1024      # Uniform batch size 
epochs = 5             # An epoch is 5 batches 
validation_split = 0.2 # Leave 20% for Validation

### Model 1: Basic RNN 

A basic RNN model is generally good baseline for sequence data.  In this model, we'll build this base RNN that translates English to French:

![RNN](images/rnn.png)

https://github.com/jvmancuso/AIND/blob/491d0b60a4d946586fd7829e3aba75ae0aca7a9c/aind2-nlp-capstone/machine_translation.ipynb

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    ## Basic Recurrent Neural Network:
    # * Accepts padded, reshaped uniform input and output sequences.
    # * Uses a connected recurrent neural network to map word IDs
    
    #print([print(input) for input in input_shape])
    
    x = Input(shape = input_shape[1:])                                        # Input : 1024 {Batch Size} x [N examples x Vocabulary x Word ID]
    z = GRU(128, return_sequences = True, dropout = 0.5)(x)                   # Gated Rectified Unit(Input) : 1024 LSTM cells that return in sequence
    y = TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))(z)  # TimeDistributed(Dense(Input))  : But run this activation on the whole time-distributed sequence, not individual words
    
    # Input/Output Association
    model = Model(x, y)
    
    # Layer Summary
    model.summary()
    
    # Compile Settings
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    # Encapsulate
    return model

In [ ]:
# Tests 
tests.test_simple_model(simple_model)

In [ ]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

# Save model, for future realoading / training resuming 
if os.path.exists(os.path.join("cache", "simple_model.h5"))== False:
    simple_rnn_model.save(os.path.join("cache", "simple_model.h5"))
else:
    simple_rnn_model = load_model(os.path.join("cache", "simple_model.h5"))
    

# Train model
simple_rnn_model.fit(tmp_x, 
                     preproc_french_sentences, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     validation_split=validation_split)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

### Model 2: Embedding 

We've turned the words into ids, but there's a better representation of a word; a **word embedding**.  

![](images/embedding.png)

An **embedding** is a **vector representation of the word** that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we'll apply this understanding to create an RNN model using embedding.

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    ## Embedded Recurrent Neural Network:
    # * Accepts padded, reshaped uniform input and output sequences
    # * Embedding those sequences into vocabulary word space
    # * Uses a connected recurrent neural network to map word IDs

    x = Input(shape = input_shape[1:])                    # Input : 1024 {Batch Size} x [N examples x Vocabulary x Word ID]
    
    z = Embedding(input_dim=english_vocab_size + 1,
                  input_length = input_shape[1],
                  output_dim=128)(x)                      # Embedding(Input): Embed the Inputs into an encoded English Vocabulary across 128 units
    
    z = GRU(units=128,
            return_sequences = True, 
            dropout = 0.5)(z)                             # Gated Rectified Unit(Input) : 128 GRU cells that return in sequence
    
    y = TimeDistributed(Dense(french_vocab_size, 
                              activation = 'softmax'))(z) # TimeDistributed(Dense(Input))  : But run this activation on the whole time-distributed sequence, not individual words
    
    # Input/Output Association
    model = Model(x, y)
    
    # Layer Summary
    model.summary()
    
    # Compile Settings
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    # Encapsulate
    return model

In [13]:
from keras.layers.embeddings import Embedding

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    
    ref: https://keras.io/layers/embeddings/
    
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # hyperparams: 
    embed_dim = 300
    rnn_dim = 64
    learning_rate = 0.01
    
    # model
    model_input = Input(shape=(input_shape[1],))
    embedding_layer = Embedding(english_vocab_size, embed_dim, input_length=input_shape[1])
    # embedding layer adds the extra dimension that was present (as just a 1) in input shape of Model 1
    rnn_input = embedding_layer(model_input)
    rnn_layer = GRU(rnn_dim, return_sequences=True)
    rnn_output = rnn_layer(rnn_input)
    tddense_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    model_output = tddense_layer(rnn_output)
    model = Model(model_input, model_output)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

In [14]:
# Test model 
tests.test_embed_model(embed_model)

In [15]:
# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Populate parameters
embed_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))

# Save model, for future realoading / training resuming 
# if os.path.exists(os.path.join("cache", "embed_model.h5"))== False:
#     embed_model.save(os.path.join("cache", "embed_model.h5"))
# else:
#     embed_model = load_model(os.path.join("cache", "embed_model.h5"))
    

# Train model
embed_model.fit(tmp_x, 
                     preproc_french_sentences, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     validation_split=validation_split)

# Print prediction(s)
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], french_tokenizer))

[[17 23  1 ... 44  0  0]
 [ 5 20 21 ... 51  2 45]
 [22  1  9 ... 34  0  0]
 ...
 [24  1 10 ... 54  0  0]
 [ 5 84  1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
Train on 110288 samples, validate on 27573 samples
Epoch 1/5
  3072/110288 [..............................] - ETA: 1:11 - loss: 5.5996 - acc: 0.3611

InvalidArgumentError: indices[479,0] = 199 is not in [0, 199)
	 [[Node: embedding_3/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_3/embeddings/read, embedding_3/Cast)]]

Caused by op 'embedding_3/Gather', defined at:
  File "/Users/v/.anaconda/envs/touristai/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/v/.anaconda/envs/touristai/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/v/.anaconda/envs/touristai/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/v/.anaconda/envs/touristai/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/v/.anaconda/envs/touristai/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/v/.a

In [ ]:
# https://github.com/jvmancuso/AIND/blob/491d0b60a4d946586fd7829e3aba75ae0aca7a9c/aind2-nlp-capstone/machine_translation.ipynb

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    ## Embedded Recurrent Neural Network:
    # * Accepts padded, reshaped uniform input and output sequences
    # * Embedding those sequences into vocabulary word space
    # * Uses a connected recurrent neural network to map word IDs
   

    x = Input(shape = input_shape[1:])                                        # Input : 1024 {Batch Size} x [N examples x Vocabulary x Word ID]
    z = Bidirectional(GRU(256, return_sequences = True, dropout = 0.5))(x)    # Bidirectional(GRU(Input)) : 256 LSTM cells that return in two reverse sequences!
    y = TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))(z)  # TimeDistributed(Dense(Input))  : But run this activation on the whole time-distributed sequence, not individual words
    
    # Input/Output Association
    model = Model(x, y)
    
    # Layer Summary
    model.summary()
    
    # Compile Settings
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    # Encapsulate
    return model

In [ ]:
# Test Model
tests.test_bd_model(bd_model)

In [ ]:
# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
bd_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))


# Save model, for future realoading / training resuming 
if os.path.exists(os.path.join("cache", "bd_model.h5"))== False:
    bd_model.save(os.path.join("cache", "bd_model.h5"))
else:
    bd_model = load_model(os.path.join("cache", "bd_model.h5"))
    

# Train model
bd_model.fit(tmp_x, 
                     preproc_french_sentences, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     validation_split=validation_split)

# Print prediction(s)
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models!  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

[**Seq2Seq!**](https://github.com/google/seq2seq) 

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # encoder
    enc_model_input = Input(shape=(input_shape[1], input_shape[2]))
    enc_rnn_layer = GRU(rnn_dim, return_state=True)
    encoded, enc_state = enc_rnn_layer(enc_model_input)
    enc_repeat = RepeatVector(output_sequence_length)
    dec_input = enc_repeat(encoded)
    
    # decoder
    dec_rnn_layer = GRU(rnn_dim, return_sequences=True)
    dec_output = dec_rnn_layer(dec_input, initial_state=enc_state)
    dec_tddense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    dec_output = dec_tddense(dec_output)
    model = Model(enc_model_input, dec_output)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return None




In [ ]:
# Test Model
tests.test_encdec_model(encdec_model)

In [ ]:
# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embed_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))


# Save model, for future realoading / training resuming 
if os.path.exists(os.path.join("cache", "embed_model.h5"))== False:
    embed_model.save(os.path.join("cache", "embed_model.h5"))
else:
    embed_model = load_model(os.path.join("cache", "embed_model.h5"))
    

# Train model
embed_model.fit(tmp_x, 
                     preproc_french_sentences, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     validation_split=validation_split)

# Print prediction(s)
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], french_tokenizer))

# OPTIONAL: Train and Print prediction(s)

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # encoder
    enc_model_input = Input(shape=(input_shape[1],))
    embedding_layer = Embedding(english_vocab_size, embed_dim, input_length=input_shape[1])
    rnn_input = embedding_layer(enc_model_input)
    enc_rnn_layer = Bidirectional(GRU(rnn_dim))
    encoded = enc_rnn_layer(rnn_input)
    bn_encoded = BatchNormalization()(encoded)
    
    # Prepare input for decoder
    enc_repeat = RepeatVector(output_sequence_length)
    dec_input = enc_repeat(bn_encoded)
    # Provide Initial State for Decoder
    # ref 1: http://nmt-keras.readthedocs.io/en/latest/tutorial.html
    # ref 2: https://github.com/keras-team/keras/issues/5971
    enc_state = Dense(rnn_dim, activation='tanh')(bn_encoded)
    
    # decoder
    dec_rnn_layer_1 = GRU(rnn_dim, return_sequences=True)
    dec_rnn_output_1 = dec_rnn_layer_1(dec_input, initial_state=enc_state)
    dec_bn_rnn_output_1 = BatchNormalization()(dec_rnn_output_1)
    
    dec_rnn_layer_2 = Bidirectional(GRU(rnn_dim, return_sequences=True))
    dec_rnn_output_2 = dec_rnn_layer_2(dec_rnn_output_1)
    dec_bn_rnn_output_2 = BatchNormalization()(dec_rnn_output_2)
    
    dec_tddense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    dec_output = dec_tddense(dec_bn_rnn_output_2)
    model = Model(enc_model_input, dec_output)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return None
tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

## Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb